In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt, seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
print(tensorflow.__version__)

NameError: ignored

In [ ]:
wine_df = pd.read_csv('winequality-red.csv')

In [ ]:
wine_df.head()

In [ ]:
wine_df.shape

In [ ]:
wine_df.info()

In [ ]:
wine_df.describe()

In [ ]:
# 2. Missing value  imputation
wine_df.isnull().sum()

In [ ]:
feature_ = wine_df.columns[:-1]
def get_percentile(feature,q_range):
    dist = wine_df[feature].describe()[str(q_range) + '%']
    return round(dist,2)
def render_counterplot():
    fig= plt.figure(figsize=(18,20))
    for column,feature in enumerate(feature_):
        fig.add_subplot(4,3,column+1)
        q1 = get_percentile(feature,25)
        q2 = get_percentile(feature,50)
        q3 = get_percentile(feature,75)
        sns.histplot(data=wine_df, x=feature, kde=True,color='orange')
        plt.axvline(q1,linestyle='--',color='green',label='Q1')
        plt.axvline(q2,linestyle='--',color='red',label='Q2')
        plt.axvline(q3,linestyle='--',color='black',label='Q3')
        plt.legend()
    plt.show()
render_counterplot()

In [ ]:
plt.figure(figsize=(18,6))

ax1= sns.countplot(x='quality',data=wine_df, palette='Set2')
plt.xlabel('Wine Quality', fontsize=14)
plt.ylabel('count', fontsize=14)

ax2 = ax1.twinx()
ax2.yaxis.tick_left()
ax1.yaxis.tick_right()
ax1.yaxis.set_label_position('right')
ax2.yaxis.set_label_position('left')
ax2.set_ylabel('Frequency [%]')

for p in ax1.patches:
    x = p.get_bbox().get_points()[:,0]
    y = p.get_bbox().get_points()[1,1]
    ax1.annotate('{:.1f}%'.format(100.*y/len(wine_df)),(x.mean(),y),ha='center')

In [ ]:
features_ = wine_df.columns.values[:-1]
fig= plt.figure(figsize=(16,26))
for column,feature in enumerate(features_):
    fig.add_subplot(5,3,column+1)
    sns.boxplot(data=wine_df,x='quality',y=feature)
plt.show()

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state=1, max_depth=12)

x = wine_df.drop(['quality'],axis=1)

rf.fit(x,wine_df.quality)

# store importance and feature names
features = wine_df.columns
importances= rf.feature_importances_
sorted_index = np.argsort(importances)[:]

plt.figure(figsize=(18,6))
plt.title('Feature Importances',fontsize=14)
plt.barh(range(len(sorted_index)),importances[sorted_index],color='green',align='center')
plt.yticks(range(len(sorted_index)),[features[i] for i in sorted_index])
plt.xlabel('feature importance',fontsize=12)
plt.show()

In [ ]:
# check for multicollinearity

plt.figure(figsize=(18,12))

sns.heatmap(x.corr(),annot=True)
plt.show()

In [ ]:
wine_df.drop(['fixed acidity','free sulfur dioxide'],axis=1,inplace= True)

In [ ]:
from imblearn.over_sampling import SMOTE
x = wine_df.drop('quality', axis = 'columns')
y = wine_df['quality']
smote = SMOTE(sampling_strategy = 'minority')
x_sm, y_sm = smote.fit_resample(x,y)


In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_sm[x_sm.columns] = scaler.fit_transform(x_sm[x_sm.columns])

In [ ]:
x_sm

In [ ]:
## need to run again for 2nd and 3 rd model as y_test was manipulated 
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test= train_test_split(x_sm,y_sm,test_size=0.2,random_state=42)

In [ ]:
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.layers import Dropout

In [ ]:
model = Sequential()
model.add(tensorflow.keras.layers.Input(shape=9,))
model.add(tensorflow.keras.layers.Dense(32,activation='relu'))
model.add(tensorflow.keras.layers.Dense(64,activation='relu'))
model.add(tensorflow.keras.layers.Dropout(0.3))
model.add(tensorflow.keras.layers.Dense(128,activation='relu'))
model.add(tensorflow.keras.layers.Dropout(0.3))
model.add(tensorflow.keras.layers.Dense(6,activation='softmax'))

In [ ]:
## need to run again for 2nd and 3 rd model as y_test was manipulated 
from sklearn.preprocessing import LabelEncoder
le= LabelEncoder()
y_train=le.fit_transform(y_train)
y_test = le.transform(y_test)

In [ ]:
y_train

In [ ]:
## need to run again for 2nd and 3 rd model as y_test was manipulated 
y_train = pd.DataFrame(y_train.reshape(len(y_train),1))
y_test = pd.DataFrame(y_test.reshape(len(y_test),1))

In [ ]:
y_train

In [ ]:
y_train

In [ ]:
## need to run again for 2nd and 3 rd model as y_test was manipulated 
y_train = tensorflow.keras.utils.to_categorical(y_train,6)
y_test = tensorflow.keras.utils.to_categorical(y_test,6)

In [ ]:
y_train

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit(x_train,y_train,epochs=400,verbose=1)

In [ ]:
y_pred = model.predict_classes(x_test)

In [ ]:
y_test

In [ ]:
y_test = np.argmax(y_test,axis=1)

In [ ]:
y_test

In [ ]:
y_pred

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test,y_pred)*100

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

In [ ]:
model = Sequential()
model.add(tensorflow.keras.layers.Input(shape=9,))
model.add(tensorflow.keras.layers.Dense(32,activation='relu'))
model.add(tensorflow.keras.layers.Dense(64,activation='relu'))
model.add(tensorflow.keras.layers.Dropout(0.3))

model.add(tensorflow.keras.layers.Dense(6,activation='softmax'))

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit(x_train,y_train,epochs=400,verbose=1)

In [ ]:
y_pred = model.predict_classes(x_test)

In [ ]:
y_test = np.argmax(y_test,axis=1)

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test,y_pred)*100

In [ ]:
model = Sequential()
model.add(tensorflow.keras.layers.Input(shape=9,))
model.add(tensorflow.keras.layers.Dense(32,activation='relu'))
model.add(tensorflow.keras.layers.Dense(64,activation='relu'))
model.add(tensorflow.keras.layers.Dropout(0.3))
model.add(tensorflow.keras.layers.Dense(128,activation='relu'))
model.add(tensorflow.keras.layers.Dropout(0.3))
model.add(tensorflow.keras.layers.Dense(256,activation='relu'))
model.add(tensorflow.keras.layers.Dropout(0.3))
model.add(tensorflow.keras.layers.Dense(6,activation='softmax'))

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit(x_train,y_train,epochs=400,verbose=1)

In [ ]:
y_pred = model.predict_classes(x_test)

In [ ]:
y_test = np.argmax(y_test,axis=1)

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test,y_pred)*100

In [ ]:
y_pred

In [ ]:
y_pred = pd.DataFrame(y_pred.reshape(len(y_test),1))

In [ ]:
y_pred

In [ ]:
x_test

In [ ]:
print(model.layers[0].get_weights()[0])
print(model.layers[0].get_weights()[1])